In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import scipy.stats
import numpy as np
import scipy.stats
import pandas as pd

In [2]:
#Modificado

boston = datasets.load_boston()

X = pd.DataFrame(data=boston.data, columns = boston['feature_names']) 
y = pd.DataFrame(boston.target)

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [3]:
#Modificado

# correlation_filtering()

# se define la funcion
def correlation_filtering(data_frame, treshold):

    df = pd.DataFrame(data_frame)
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # guarda las columnas que se van a tirar
    to_drop =[]

    for n in range(df.shape[1]): # itera sobre columnas del data frame
        
        if any(upper.iloc[:,n]>treshold):     # si algun elemento de la matriz de correlacion de la columna n es mayor al treshold
            print(upper.columns[n], " Sobrepasa el umbral")
            to_drop.append(df.columns[n])     # se agrega para borrarse
            
    print('columns dropped: ', to_drop)            # columnas que se van a tirar   

    # Drop features 
    df.drop(columns= to_drop,inplace=True)

    return df

In [4]:
fcf = correlation_filtering(X_train, 0.7)
fcf

NOX  Sobrepasa el umbral
AGE  Sobrepasa el umbral
DIS  Sobrepasa el umbral
TAX  Sobrepasa el umbral
columns dropped:  ['NOX', 'AGE', 'DIS', 'TAX']


,CRIM,ZN,INDUS,CHAS,RM,RAD,PTRATIO,B,LSTAT
482,5.73116,0.0,18.10,0.0,7.061,24.0,20.2,395.28,7.01
117,0.15098,0.0,10.01,0.0,6.021,6.0,17.8,394.51,10.30
103,0.21161,0.0,8.56,0.0,6.137,5.0,20.9,394.47,13.44
415,18.08460,0.0,18.10,0.0,6.434,24.0,20.2,27.25,29.05
149,2.73397,0.0,19.58,0.0,5.597,5.0,14.7,351.85,21.45
...,...,...,...,...,...,...,...,...,...
336,0.03427,0.0,5.19,0.0,5.869,5.0,20.2,396.90,9.80
51,0.04337,21.0,5.64,0.0,6.115,4.0,16.8,393.97,9.43
189,0.08370,45.0,3.44,0.0,7.185,5.0,15.2,396.90,5.39
77,0.08707,0.0,12.83,0.0,6.140,5.0,18.7,386.96,10.27


In [5]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

#X = pd.DataFrame(data=boston.data, columns = boston['feature_names']) 

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [6]:
# Fast Correlation Based filtering

dfx = pd.DataFrame(X) # base que NO incluye la variable objetivo
 
# 1. Obtener la correlación de cada variable en X con la variable de salida y guardar las que pasen el umbral
treshold = 0.5
corr_xy = [] # aquí se van a guardar las correlaciones de cada xi con y
high_corr = [] # aquí se guardan las variables que están por arriba del umbral

for n in range(dfx.shape[1]): # itera por columnas del data frame
    column = dfx.iloc[:,n]    # guarda la columna
    a = abs(np.corrcoef(column, y)[0,1]) # calcula la correlacion en valor absoluto
    corr_xy.append(a)                
    if a > treshold:                     
        high_corr.append(dfx.columns[n])
    
print(corr_xy) 
print(high_corr)

# 2. Ordenar las variables de mayor a menor correlación con Y
temp = pd.DataFrame(corr_xy).reset_index()
temp = temp.rename(columns={0: 'corr'})
temp = temp.sort_values(by=['corr'], ascending=False)
print(temp)  

#3. Medir la correlación de la mejor variable con las demás
feature = temp.iloc[0,0] # Característica xi más correlacionado con y
df_xi = dfx[feature] # Dataframe con el xi más correlacionado con y 
df_witouth_xi = pd.DataFrame(dfx)
df_witouth_xi.drop(columns = feature ,inplace=True) # Dataframe sin el xi más correlacionado con y

corr_xi_all = abs(df_witouth_xi.corrwith(df_xi)) # Correlacion de xi con las demás características
corr_xi_all

# 4. Tirar variables que estén muy correlacionadas con la primera variable.
to_drop =[] # vector de variables que se van a borrar

for n in range(df_witouth_xi.shape[1]): # itera sobre columnas del data frame
        
        if (corr_xi_all[n]>treshold):     # si algun elemento de la matriz de correlacion de la columna n es mayor al treshold
            print(corr_xi_all[n], " sobrepasa el umbral")
            to_drop.append(dfx.columns[n])     # se agrega para borrarse
            
print('columns dropped: ', to_drop)            # columnas que se van a tirar   

# Drop features 
df_witouth_xi.drop(columns= to_drop,inplace=True)

# 5. Unir DF
result = pd.concat([df_xi, df_witouth_xi], axis=1, sort=False)
result

[0.3883046085868115, 0.36044534245054344, 0.48372516002837357, 0.17526017719029863, 0.4273207723732825, 0.6953599470715393, 0.37695456500459623, 0.24992873408590388, 0.38162623063977763, 0.46853593356776685, 0.5077866855375616, 0.33346081965706653, 0.7376627261740151]
[5, 10, 12]
    index      corr
12     12  0.737663
5       5  0.695360
10     10  0.507787
2       2  0.483725
9       9  0.468536
4       4  0.427321
0       0  0.388305
8       8  0.381626
6       6  0.376955
1       1  0.360445
11     11  0.333461
7       7  0.249929
3       3  0.175260
0.6037997164766219  sobrepasa el umbral
0.5908789208808461  sobrepasa el umbral
0.6138082718663953  sobrepasa el umbral
0.6023385287262402  sobrepasa el umbral
0.5439934120015699  sobrepasa el umbral
columns dropped:  [2, 4, 5, 6, 9]


,12,0,1,3,7,8,10,11
0,4.98,0.00632,18.0,0.0,4.0900,1.0,15.3,396.90
1,9.14,0.02731,0.0,0.0,4.9671,2.0,17.8,396.90
2,4.03,0.02729,0.0,0.0,4.9671,2.0,17.8,392.83
3,2.94,0.03237,0.0,0.0,6.0622,3.0,18.7,394.63
4,5.33,0.06905,0.0,0.0,6.0622,3.0,18.7,396.90
...,...,...,...,...,...,...,...,...
501,9.67,0.06263,0.0,0.0,2.4786,1.0,21.0,391.99
502,9.08,0.04527,0.0,0.0,2.2875,1.0,21.0,396.90
503,5.64,0.06076,0.0,0.0,2.1675,1.0,21.0,396.90
504,6.48,0.10959,0.0,0.0,2.3889,1.0,21.0,393.45


In [71]:
# Fast Correlation Based filtering v2

boston = datasets.load_boston()

X = boston.data
y = boston.target
X = pd.DataFrame(data=boston.data, columns = boston['feature_names']) 

dfx = pd.DataFrame(X) # base que NO incluye la variable objetivo
#print(dfx)
 
# 1. Obtener la correlación de cada variable en X con la variable de salida y guardar las que pasen el umbral
treshold = 0.5
corr_xy = [] # aquí se van a guardar las correlaciones de cada xi con y
high_corr = [] # aquí se guardan las variables que están por arriba del umbral

for n in range(dfx.shape[1]): # itera por columnas del data frame
    column = dfx.iloc[:,n]    # guarda la columna
    a = abs(np.corrcoef(column, y)[0,1]) # calcula la correlacion en valor absoluto
    corr_xy.append(a)                
    if a > treshold:                     
        high_corr.append(dfx.columns[n])
        
corr_xy = np.array(corr_xy)
corr_xy = corr_xy.reshape(1, corr_xy.shape[0]) 
corr_xy = pd.DataFrame(data = corr_xy, columns = dfx.columns)
  
print("corr_xy: ",corr_xy) 
print("high_corr: ",high_corr)

# 2. Ordenar las variables de mayor a menor correlación con Y
temp = pd.DataFrame(corr_xy).reset_index()
temp = temp.rename(columns={0: 'corr'})
temp = temp.sort_values(by=['corr'], ascending=False)
print(temp)  

#3. Medir la correlación de la mejor variable con las demás
feature = temp.iloc[0,0] # Característica xi más correlacionado con y
df_xi = dfx[feature] # Dataframe con el xi más correlacionado con y 
df_witouth_xi = pd.DataFrame(dfx)
df_witouth_xi.drop(columns = feature ,inplace=True) # Dataframe sin el xi más correlacionado con y

corr_xi_all = abs(df_witouth_xi.corrwith(df_xi)) # Correlacion de xi con las demás características
corr_xi_all

# 4. Tirar variables que estén muy correlacionadas con la primera variable.
to_drop =[] # vector de variables que se van a borrar

for n in range(df_witouth_xi.shape[1]): # itera sobre columnas del data frame
        
        if (corr_xi_all[n]>treshold):     # si algun elemento de la matriz de correlacion de la columna n es mayor al treshold
            print(corr_xi_all[n], " sobrepasa el umbral")
            to_drop.append(dfx.columns[n])     # se agrega para borrarse
            
print('columns dropped: ', to_drop)            # columnas que se van a tirar   

# Drop features 
df_witouth_xi.drop(columns= to_drop,inplace=True)

# 5. Unir DF
result = pd.concat([df_xi, df_witouth_xi], axis=1, sort=False)
result

corr_xy:         CRIM        ZN     INDUS     CHAS       NOX       RM       AGE  \
0  0.388305  0.360445  0.483725  0.17526  0.427321  0.69536  0.376955   

        DIS       RAD       TAX   PTRATIO         B     LSTAT  
0  0.249929  0.381626  0.468536  0.507787  0.333461  0.737663  
high_corr:  ['RM', 'PTRATIO', 'LSTAT']


KeyError: 'corr'

In [41]:
column = dfx.iloc[:,0]    # guarda la columna
column2 = pd.DataFrame(column)

a = abs(np.corrcoef(column, y)[0,1]) # calcula la correlacion en valor absoluto

b = column2.corrwith(pd.DataFrame(y))

c = scipy.stats.pearsonr(column, y)

corr_xy.append(a)                
if a > treshold:                     
    high_corr.append(dfx.columns[n])
    
high_corr
corr_xy

[0.3883046085868115,
 0.36044534245054344,
 0.48372516002837357,
 0.17526017719029863,
 0.4273207723732825,
 0.6953599470715393,
 0.37695456500459623,
 0.24992873408590388,
 0.38162623063977763,
 0.46853593356776685,
 0.5077866855375616,
 0.33346081965706653,
 0.7376627261740151,
 0.3883046085868115,
 0.3883046085868115,
 0.3883046085868115,
 0.3883046085868115,
 0.3883046085868115,
 0.3883046085868115,
 0.3883046085868115,
 0.3883046085868115,
 0.3883046085868115]

In [46]:
tt= pd.DataFrame(corr_xy)
tt

,0
0,0.388305
1,0.360445
2,0.483725
3,0.175260
4,0.427321
5,0.695360
6,0.376955
7,0.249929
8,0.381626
9,0.468536


In [73]:
tt.transpose

<bound method DataFrame.transpose of            0
0   0.388305
1   0.360445
2   0.483725
3   0.175260
4   0.427321
5   0.695360
6   0.376955
7   0.249929
8   0.381626
9   0.468536
10  0.507787
11  0.333461
12  0.737663
13  0.388305
14  0.388305
15  0.388305
16  0.388305
17  0.388305
18  0.388305
19  0.388305
20  0.388305
21  0.388305>

In [68]:
dfx.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [ ]:
####MODIFICADO

boston = datasets.load_boston()

X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X,y)

# Fast Correlation Based filtering

dfx = pd.DataFrame(X) # base que NO incluye la variable objetivo

# 1. Obtener la correlación de cada variable en X con la variable de salida y guardar las que pasen el umbral
treshold = 0.5
corr_xy = [] # aquí se van a guardar las correlaciones de cada xi con y
high_corr = [] # aquí se guardan las variables que están por arriba del umbral

for n in range(dfx.shape[1]): # itera por columnas del data frame
    column = dfx.iloc[:,n]    # guarda la columna
    corr = scipy.stats.pearsonr(column, y)  # Calcula Correlación
    a = abs(corr[0]) # Correlación en valor absoluto
    corr_xy.append(a)                
    if a > treshold:                     
        high_corr.append(dfx.columns[n])
    
print(corr_xy) 
print(high_corr)

# 2. Ordenar las variables de mayor a menor correlación con Y
temp = pd.DataFrame(corr_xy).reset_index()
temp = temp.rename(columns={0: 'corr'})
temp = temp.sort_values(by=['corr'], ascending=False)
print(temp)